In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
from sklearn.linear_model import LogisticRegression
import keras 
from keras.models import Sequential
from keras.layers import Dense

%matplotlib inline


Using TensorFlow backend.
/home/romulus/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/romulus/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/romulus/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/romulus/.local/lib/python3.5/site-packages/tensorflow/py

AttributeError: module 'tensorflow.python.keras.backend' has no attribute 'get_graph'

In [ ]:
col_names=['PassengerId',
           'Survived',
           'Pclass',
           'Name',
           'Sex',
           'Age', 
           'SibSp',
           'Parch',
           'Ticket',
           'Fare',
           'Cabin',
           'Embarked']

In [ ]:
#ds=pd.read_csv('train.csv', ',', names=col_names, index_col=False)
ds = pd.read_csv('train.csv')
test_ds = pd.read_csv('test.csv')
ds.head()

In [ ]:
ds.shape

In [ ]:
ds.info()

In [ ]:
ds['Sex'].replace(['female','male'], [0,1], inplace = True)
test_ds['Sex'].replace(['female','male'], [0,1], inplace = True)

In [ ]:
drop_names = ['PassengerId',
              'Name',
              'Ticket',
              'Fare',
              'Embarked',
              'Cabin']
drop_names_test = ['Name',
                   'Ticket',
                   'Fare',
                   'Embarked',
                   'Cabin']

In [ ]:
ds.fillna(ds.mean(), inplace = True)
test_ds.fillna(test_ds.mean(), inplace = True)

In [ ]:
ds = ds.drop(columns = drop_names)
test_ds = test_ds.drop(columns = drop_names_test)
test_ds.head()

In [ ]:
ds.describe()

In [ ]:
test_ds.describe()

In [ ]:
ds.Age.hist()

In [ ]:
ds.corr()

In [ ]:
import seaborn as sns
corr=ds.corr()
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, cmap=sns.diverging_palette(220, 10, as_cmap=True))

In [ ]:
ds[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
ds[['Age', 'Survived']].groupby(['Age'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
ds[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
ds[['Parch', 'Survived']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
ds[['SibSp', 'Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
g = sns.FacetGrid(ds, col='Survived')
g.map(plt.hist, 'Age', bins=20)

In [ ]:
g = sns.FacetGrid(ds, col='Survived')
g.map(plt.hist, 'Pclass', bins=20)

In [ ]:
grid = sns.FacetGrid(ds, col='Survived', row='Pclass', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

In [ ]:

X_train = ds.drop("Survived", axis=1)
Y_train = ds["Survived"]
X_test = test_ds.drop(columns = 'PassengerId')
#X_test  = ds.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

In [ ]:
ds.head()

In [ ]:
X_test.head()

In [ ]:
X_test.tail()

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

In [ ]:
pd.DataFrame({
        "PassengerId": test_ds["PassengerId"],
        "Survived": Y_pred
    }).to_csv('submission.csv', index=False)

In [ ]:
model = Sequential()

model.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu', input_dim = 17))
model.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Train the ANN
model.fit(X_train, y_train, batch_size = 32, epochs = 200)
